# **Настройки для Colab**

In [7]:
import nest_asyncio
nest_asyncio.apply()

# **Установка необходимых библиотек**

In [8]:
!pip install -r requirements.txt
# !pip install langchain-core==0.3.79 langgraph==0.6.11 langchain-huggingface==0.3.1 langchain-cohere==0.4.6 langchain-groq==0.3.8 chromadb ddgs playwright-stealth crawl4ai nbclient jupyter-client rich
# !pip install llama-index llama-index-vector-stores-chroma llama-index-embeddings-huggingface llama-index-llms-langchain
# !playwright install

# !pip install --upgrade "mlflow[databricks]>=3.1"

In [ ]:
# !pip freeze > requirements.txt

# Настройки для тестирования

In [9]:
# Кэширование вызовов LLM
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
set_llm_cache(InMemoryCache())

# **Импорт необходимых утилит. Инициализация LLM-модели.**

In [10]:
"""
Конфигурафия LlamaIndex.
Важен порядок инициализации, иначе возикнет ошибка,
связанная с API ключом OpenAI. Это связано с формой наследования.
Подробнее: https://github.com/run-llama/llama_index/issues/13793
"""
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from llama_index.llms.langchain import LangChainLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

"""
Список моделей Cohere https://docs.cohere.com/docs/models
command-a-03-2025 command-r7b-12-2024 command-a-reasoning-08-2025
command-r-08-2024 command-r-plus-08-2024
embed-v4.0 embed-multilingual-v3.0
"""
llm = init_chat_model(
    model="command-a-03-2025",
    model_provider="cohere",
    temperature=0.1,
)
# llm = HuggingFaceEndpoint(
#     endpoint_url="openai/gpt-oss-20b",
#     task="text-generation",
#     temperature=0.1,
# )
# llm = ChatHuggingFace(llm=llm)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

from llama_index.core import Settings
Settings.llm = LangChainLLM(llm)
Settings.embed_model = embed_model

In [11]:
# Отладка и трассировка
import mlflow

# LLM
from langchain_huggingface import (
    HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings)
from langchain_cohere import ChatCohere, CohereEmbeddings
from cohere.errors import TooManyRequestsError

# RAG
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.core.base.base_query_engine import BaseQueryEngine
from llama_index.vector_stores.chroma import ChromaVectorStore

# Построение графа
from typing_extensions import Annotated, Literal
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_core.runnables.config import RunnableConfig
# from langgraph.graph import MessagesState

# Написание подсказок
from langchain_core.prompts import (
    SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder)

# Для работы с jupiter файлом
from jupyter_client import KernelManager, BlockingKernelClient
import nbformat
from nbformat import NotebookNode
from nbformat.v4 import new_code_cell, new_markdown_cell

# Для импорта API ключей
from dotenv import load_dotenv
import os

# Работа с моделями
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import (
    AnyMessage, HumanMessage, AIMessage,
    ToolMessage, BaseMessage, SystemMessage, RemoveMessage)

# Извлечение данных с сайтов
from ddgs import DDGS
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig
from crawl4ai.async_configs import CrawlerRunConfig, CacheMode

# Описание инструментов и их вызов
import uuid
import json
from json import JSONDecodeError
from typing import Type, List, Optional, Any, Iterable, Dict, TypedDict
from pydantic import BaseModel, Field, ConfigDict
from langchain_core.tools import BaseTool, ToolException, InjectedToolArg
from langgraph.prebuilt import InjectedStore, ToolNode

# Реализация памяти агента
import chromadb
from chromadb import QueryResult, GetResult, Collection, EmbeddingFunction
from chromadb.config import Settings
import chromadb.utils.embedding_functions as embedding_functions
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_core.documents import Document (если перепишу на LlamaIndex, то удалить)
from langgraph.config import get_store, get_config
from langgraph.store.base import BaseStore, Result, Op
from langgraph.checkpoint.memory import InMemorySaver # Кратковременная

# Читаемый вывод
from rich import print
from rich.syntax import Syntax
from rich.console import Console

# Включение отладки

In [12]:
mlflow.langchain.autolog()

# **Создание хранилища памяти агента**

In [13]:
class ChromaStore(BaseStore):
  def __init__(
      self,
      embedding_function: EmbeddingFunction = embedding_functions.DefaultEmbeddingFunction(),
      persist_directory: str = "/content"
  ):
    self.client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(anonymized_telemetry=False)
    )
    self.embedding_fn = embedding_function

  def _get_collection(self, namespace: tuple[str, ...]) -> Collection:
    name = ".".join(namespace)
    return (
        self.client.get_or_create_collection(
            name,
            embedding_function=self.embedding_fn
        )
    )

  def get(self, namespace: tuple[str, ...], key: str) -> Dict | None:
    """ Извлекает один элемент. """
    collection: Collection = self._get_collection(namespace)
    results: GetResult = collection.get(ids=[key])
    if results["documents"]:
      return {
          "document": results["documents"][0],
          "metadata": results["metadatas"][0],
      }
    return None

  def search(
      self,
      namespace: tuple[str, ...],
      query: str | None = None,
      filter: dict[str, Any] | None = None,
      limit: int = 10,
  ) -> List[Dict]:
    """ Выполняет поиск элементов в пределах пространства имен. """
    collection: Collection = self._get_collection(namespace)
    if query is None:
      return []
    results: QueryResult = collection.query(
        query_texts=[query],
        n_results=limit,
        where=filter or None
    )
    matches = []
    for doc, metadata, doc_id in zip(
        results["documents"][0], results["metadatas"][0], results["ids"][0]
    ):
      matches.append({
          # "key": doc_id,
          "text": doc,
          "metadata": metadata
      })
    return matches

  def put(
      self,
      namespace: tuple[str, ...],
      key: str,
      value: dict[str, Any]
  ) -> None:
    """ Сохраняет или обновляет элемент в хранилище. """
    collection: Collection = self._get_collection(namespace)
    docs = value.get("document")
    metds = value.get("metadata")
    collection.upsert(ids=[key], documents=docs, metadatas=metds)

  def delete(self, namespace: tuple[str, ...], key: str) -> None:
    """ Удаляет элемент. """
    collection: Collection = self._get_collection(namespace)
    collection.delete(ids=[key])

  def abatch(self, ops: Iterable[Op]) -> List[Result]:
    pass

  def batch(self, ops: Iterable[Op]) -> List[Result]:
    pass

# **Инициализация векторного хранилища и делителя файлов.**

In [14]:
# RAG
db = chromadb.PersistentClient(
    path="/content",
    settings=Settings(anonymized_telemetry=False)
)
chroma_collection = db.get_or_create_collection("web_docs")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Долговременная память
store = ChromaStore(persist_directory="/content")


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
#     add_start_index=True,
# )


# **Создание утилит для работы с Jupyter Notebook**

Матириалы для изучения проектирования инструментов
1) https://python.langchain.com/docs/how_to/custom_tools/#returning-artifacts-of-tool-execution
2) https://python.langchain.com/docs/integrations/tools/

## **Создание хранилища для объекта Jupyter Notebook и Jupyter Kernel**

In [15]:
class ArtifactStore:
  def __init__(self):
    self._path_file_jupiter_notebook: dict[str, str] = dict()
    self._notebooks: dict[str, NotebookNode] = dict()
    self._kernels: dict[str, KernelManager] = dict()
    self._clients: dict[str, BlockingKernelClient] = dict()

  def save_notebook(self, path_file: str, notebook: NotebookNode) -> str:
    notebook_id = str(uuid.uuid4())
    self._path_file_jupiter_notebook[notebook_id] = path_file
    self._notebooks[notebook_id] = notebook
    return notebook_id

  def get_notebook(self, notebook_id: str) -> NotebookNode:
    return self._notebooks[notebook_id]

  def get_notebook_path_file(self, notebook_id: str) -> str:
    return self._path_file_jupiter_notebook[notebook_id]

  def save_kernel_manager(self, kernel: KernelManager) -> str:
    kernel_id = str(uuid.uuid4())
    self._kernels[kernel_id] = kernel
    return kernel_id

  def get_kernel_manager(self, kernel_id: str) -> KernelManager:
    return self._kernels[kernel_id]

  def save_client(self, client: BlockingKernelClient) -> str:
    client_id = str(uuid.uuid4())
    self._clients[client_id] = client
    return client_id

  def get_client(self, client_id: str) -> BlockingKernelClient:
    return self._clients[client_id]


artifact_store = ArtifactStore()

## **Утилита создания файла ноутбука**

In [16]:
class CreateNotebookInput(BaseModel):
  file_name: str = Field(description=("Строка. "
      "Имя файла Jupyter Notebook (с расширением .ipynb или без него)."))
  path: str = Field(description=("Строка. "
  "Путь к директории, где должен быть создан файл Jupyter Notebook."))


class CreateNotebookTool(BaseTool):
  name: str = "creating_a_jupyter_notebook_file"
  description: str = (
      "Инструмент необходим если пользователю "
      "нужен новый Jupyter Notebook.\n"
      "После создания можно загрузить Jupyter Notebook в память "
      "и выполнять работу с его ячейками.\n"
      "Создает новый файл Jupyter Notebook (.ipynb) "
      "по указанному пути.\n\n"
      "Параметры:\n"
      "- `file_name`: имя Jupyter Notebook "
      "(можно указать с расширением или без);\n"
      "- `path`: путь к директории, где будет создан файл.\n\n"
      "Если файл уже существует, будет вызвана ошибка.")
  args_schema: Type[BaseModel] = CreateNotebookInput
  return_direct: bool = False

  def _run(self, file_name: str, path: str) -> str:
    try:
      if not os.path.isdir(path):
        raise FileNotFoundError(
            f"The specified path `{path}` does not exist.")
      if not path.endswith("/"):
        path += "/"
      if not file_name.endswith(".ipynb"):
        file_name += ".ipynb"
      full_path: str = os.path.join(path, file_name)
      if not os.path.isfile(full_path):
        nb: NotebookNode = nbformat.v4.new_notebook()
        with open(full_path, 'w') as f:
          nbformat.write(nb, f)
        return f"Файл {file_name} успешно создан в папке {path}."
      else:
        raise FileExistsError("File with that name already exists.")
    except Exception as exc:
      raise ToolException(
          f"Error: couldn't create notepad along the way `{full_path}`.\n"
          f"Reason: {str(exc)}"
      )

In [17]:
# create_jupyter_notebook = CreateNotebookTool()

# print(create_jupyter_notebook.name)
# print(create_jupyter_notebook.description)
# print(create_jupyter_notebook.args)
# print(create_jupyter_notebook.args_schema)
# print(create_jupyter_notebook.return_direct)

# create_jupyter_notebook.invoke({"file_name": "test_notebook", "path": "/content"})

## **Утилита загрузки файла ноутбука**

In [18]:
class LoadNotebookInput(BaseModel):
  path: str = Field(description=(
      "Строка. Полный путь к файлу Jupyter Notebook "
      "(с расширением .ipynb), который нужно загрузить в память."))


class LoadNotebookResult(BaseModel):
  notebook_id: str = Field(description=(
      "Строка формата UUID. "
      "Уникальный идентификатор загруженного Jupyter Notebook, "
      "используемый для обращения к нему другими инструментами."))


class LoadJupyterNotebookTool(BaseTool):
  name: str = "load_jupyter_notebook"
  description: str = (
      "## Назначение\n"
      "Инструмент необходим для "
      "выполнения действий с Jupyter Notebook или его ячейками. "
      "Например добавить/удалить/изменить/выполнить ячейки "
      "или сохранить изменения.\n\n"
      "## Что делает\n"
      "Инструмент загружает Jupyter Notebook "
      "(.ipynb файл) с диска в память.\n\n"
      "## Входные данные\n"
      "- `path`: Путь к файлу Jupyter Notebook.\n\n"
      "## Возвращаемые данные\n"
      "- `notebook_id`: Уникальный идентификатор Jupyter Notebook."
  )
  args_schema: Type[BaseModel] = LoadNotebookInput
  return_direct: bool = False
  # response_format: str = "content_and_artifact"

  def _run(self, path: str) -> LoadNotebookResult:
    try:
      with open(path, "r", encoding="utf-8") as f:
        notebook = nbformat.read(f, as_version=4)

      notebook_id = artifact_store.save_notebook(
          path_file=path,
          notebook=notebook)

      # content: str = (
      #     "Файл успешно загружен. "
      #     f"Его идентификатор notebook_id: {notebook_id}."
      # )
      artifact: LoadNotebookResult = LoadNotebookResult(
          notebook_id=notebook_id
      ).model_dump_json()
      return artifact
      # return content, artifact

    except Exception as exc:
      raise ToolException(
          f"Error: Failed to load notebook from path `{path}`.\n"
          f"Reason: {str(exc)}"
      )

In [19]:
# load_jupyter_notebook = LoadJupyterNotebookTool()

# print(load_jupyter_notebook.name)
# print(load_jupyter_notebook.description)
# print(load_jupyter_notebook.args)
# print(load_jupyter_notebook.return_direct)

# load_jupyter_notebook.invoke({"path": "/content/test_notebook_for_llm.ipynb"})

## **Утилита запуска Jupyter Kernel**

In [20]:
class BootJupyterKernelInput(BaseModel):
  pass

class BootJupyterKernelResult(BaseModel):
    kernel_manager_id: str = Field(description=(
        "Строка формата UUID. Уникальный идентификатор менеджера ядра Jupyter, "
        "связанного с запущенным ядром."
        )
    )

    client_id: str = Field(description=(
        "Строка формата UUID. Уникальный идентификатор клиента Jupyter, "
        "подключённого к этому ядру."
        )
    )

class BootJupyterKernelTool(BaseTool):
  name: str = "booting_jupyter_kernel"
  description: str = (
      "Запускает новое ядро Jupyter, необходимое для выполнения Python-кода. "
      "Используйте этот инструмент, "
      "когда нужно начать сессию выполнения кода в Jupyter Notebook. "
      "Возвращает уникальные идентификаторы:\n"
      "- `kernel_manager_id`: строка формата UUID. "
      "Уникальный идентификатор менеджера ядра;\n"
      "- `client_id`: строка формата UUID. "
      "Уникальный идентификатор клиента Jupyter."
  )
  args_schema: Type[BaseModel] = BootJupyterKernelInput
  return_direct: bool = False
  # response_format: str = "content_and_artifact"

  def _run(self) -> BootJupyterKernelResult:
    try:
      km = KernelManager()
      km.start_kernel()
      kc = km.client()
      kc.start_channels()
      kc.wait_for_ready()

      artifact: BootJupyterKernelResult = BootJupyterKernelResult(
          kernel_manager_id=artifact_store.save_kernel_manager(km),
          client_id=artifact_store.save_client(kc),
      )
      # content: str = (
      #     "Ядро успешно загружено.\n",
      #     f"kernel_manager_id: {artifact.kernel_manager_id}\n",
      #     f"client_id: {artifact.client_id}",
      # )
      # return content, artifact.model_dump_json()
      return artifact.model_dump_json()
    except Exception as exc:
      raise ToolException(f"Error: Failed to boot Jupyter kernel.\n{str(exc)}")

In [21]:
# boot_jupiter = BootJupyterKernelTool()

# print(boot_jupiter.name)
# print(boot_jupiter.description)
# print(boot_jupiter.args)
# print(boot_jupiter.return_direct)

# boot_jupiter.invoke({})

## **Утилита загрузки/создания ноутбука**

In [22]:
class OpenOrCreateNotebookTool(BaseTool):
    name: str = "open_or_create_notebook"
    description: str = (
        "Открывает Jupyter Notebook по имени файла. "
        "Если файл существует — он загружается. "
        "Если не существует — создаётся новый блокнот и загружается. "
        "Параметры:\n"
        "- `file_name`: имя файла блокнота, например 'my_notebook.ipynb' "
        "(можно указать с расширением или без);\n"
        "- `path`: путь к директории расположения файла.\n\n"
        "Возвращает `notebook_id` — cтрока формата UUID, "
        "являющуюся уникальным идентификатором загруженного блокнота."
    )
    args_schema: Type[BaseModel] = CreateNotebookInput
    return_direct: bool = False
    # response_format: str = "content_and_artifact"

    def _run(self, file_name: str, path: str
             ) -> str:
            #  ) -> LoadNotebookResult:
        try:
          full_path: str = os.path.join(path, file_name)
          if not os.path.exists(full_path):
            # Создание нового блокнота
            res_create_notebook = CreateNotebookTool().invoke({
                "file_name": file_name, "path": path})
          # Загрузка блокнота
          # res_load, artifact_load =
          load_result = LoadJupyterNotebookTool().invoke({
              "path": full_path})
          notebook_id: str = json.loads(load_result)["notebook_id"]

          # Запуск ядра
          result_loading_kernel = BootJupyterKernelTool().invoke({})
          client_id: str = json.loads(
              result_loading_kernel)["client_id"]
          kernel_manager_id: str = json.loads(
              result_loading_kernel)["kernel_manager_id"]

          content = (
              "Блокнот успешно загружен.\n"
              "Ядро запущено.\n"
              "Были получены следующие идентификаторы:\n"
              f"'notebook_id': {notebook_id},\n"
              f"'client_id': {client_id},\n"
              f"'kernel_manager_id': {kernel_manager_id}"
          )
          # artifact = artifact_load
          return content

        except Exception as exc:
            raise ToolException(
                f"Error: couldn't open or create notebook. {str(exc)}")


In [23]:
# open_or_create_jupiter = OpenOrCreateNotebookTool()

# print(open_or_create_jupiter.name)
# print(open_or_create_jupiter.description)
# print(open_or_create_jupiter.args)
# print(open_or_create_jupiter.return_direct)

# open_or_create_jupiter.invoke({
#     "file_name": "my_notebook.ipynb", "path": "/content"})

## **Утилита выполнения ячеек блокнота**

In [24]:
class ExecuteNotebookCellsInput(BaseModel):
  cell_indexes: List[int] = Field(description=(
      "Список номеров ячеек, "
      "которые необходимо выполнить в Jupyter Notebook."))
  notebook_id: str = Field(description=(
      "Строка формата UUID. Уникальный идентификатор Jupyter Notebook."))
  client_id: Optional[str] = Field(description=(
      "Строка формата UUID. Уникальный идентификатор клиента Jupyter."))


class ExecuteNotebookCellsResult(BaseModel):
  logs: str = Field(description=(
      "Строка с логами выполнения указанных ячеек. "
      "Для каждой ячейки указывается её исходный код, "
      "результат выполнения, а также возможные "
      "ошибки (если они возникли)."
)
  )


class ExecuteNotebookCellsTool(BaseTool):
  name: str = "execute_cells_in_jupyter_notebook"
  description: str = (
      "Выполняет указанные ячейки Jupyter Notebook по их индексам."
      "Параметры:\n"
      "- `cell_indexes`: список номеров ячеек;\n"
      "- `notebook_id`: строка формата UUID. "
      "Уникальный идентификатор Jupyter Notebook;\n"
      "- `client_id`: строка формата UUID. "
      "Уникальный идентификатор клиента Jupyter.\n"
      "Возвращает текстовую строку:\n"
      "- `logs`: строка с логами выполнения кода из ячеек: "
      "исходный код, вывод и возможные ошибки."
  )
  args_schema: Type[BaseModel] = ExecuteNotebookCellsInput
  return_direct: bool = False
  response_format: str = "content_and_artifact"

  def _run(self, cell_indexes: List[int], notebook_id: str,
           client_id: str) -> ExecuteNotebookCellsResult:
    try:
      lines: list[str] = []
      notebook = artifact_store.get_notebook(notebook_id)
      client = artifact_store.get_client(client_id)

      for i in cell_indexes:
        if len(notebook.cells) == 0:
          raise ToolException(
              "В блокноте нет ячеек. "
              "Добавьте ячейки и повторите попытку снова."
          )
        if not( 0 <= i < len(notebook.cells) ):
          raise ToolException(
              "Индекс ячейки вне допустимого диапазона."
              "Уточните количество ячеек, чтобы указать верные индексы."
          )
        cell = notebook.cells[i]
        if cell.cell_type != "code":
          continue

        lines.append(f"\n# Выполнение ячейки №{i}\n{cell.source}\n")
        msg_id: str = client.execute(cell.source)

        while True:
          msg = client.get_iopub_msg(timeout=60)

          if (
              "parent_header" in msg and
              msg["parent_header"].get("msg_id") != msg_id
          ):
            continue

          if msg["msg_type"] == "stream":
            lines.append(f"\n## Вывод ячейки №{i}:\n")
            lines.append(msg["content"].get("text", ""))
          elif msg["msg_type"] == "error":
            lines.append(f"\n## Ошибка в ячейке №{i}:\n")
            lines.append(msg["content"]["evalue"])
          elif (
                msg["msg_type"] == "status" and
                msg["content"]["execution_state"] == "idle"
          ):
            break

      return "Ячейки успешно выполнены", ExecuteNotebookCellsResult(
          logs="\n\n".join(lines)
          ).model_dump_json()

    except Exception as exc:
      raise ToolException(f"Error: Failed to execute cells.\n{str(exc)}")

In [25]:
# execute_cell = ExecuteNotebookCellsTool()

# print(execute_cell.name)
# print(execute_cell.description)
# print(execute_cell.args)
# print(execute_cell.return_direct)

# execute_cell.invoke(
#     {'cell_indexes': [0], 'notebook_id':
# '9ba651a0-cd80-4687-9a53-ac6480efc911', 'client_id':
# '759a6a6a-2bd8-4b39-b5f8-d2d1ca118316'}
#     )

## **Утилита выполнения ячеек с возможность автоматической дозагрузки ядра Jupyter**

In [26]:
class SmartExecuteCellsTool(BaseTool):
    name: str = "execute_cells"
    description: str = (
        "Выполняет указанные ячейки Jupyter Notebook. "
        "Если ядро ещё не запущено — запускает его автоматически. "
        "Внутри вызывает booting_jupyter_kernel и "
        "execute_cells_in_jupyter_notebook."
        "Параметры:\n"
        "- `cell_indexes`: список номеров ячеек;\n"
        "- `notebook_id`: строка формата UUID. "
        "Уникальный идентификатор Jupyter Notebook;\n"
        "- `client_id`: строка формата UUID. "
        "Уникальный идентификатор клиента Jupyter (опционально).\n\n"
        "Важное замечание!\n"
        "Значения notebook_id и client_id "
        "должны отличаться друг от друга!\n"
        "В случае сомнений необходимо вызвать "
        "инструмент без параметра client_id или вызвать "
        "инструмент booting_jupyter_kernel!\n\n"
        "Возвращает текстовую строку:\n"
        "- `logs`: логи выполнения кода из ячеек: "
        "исходный код, вывод и возможные ошибки."
    )
    args_schema: Type[BaseModel] = ExecuteNotebookCellsInput
    return_direct: bool = False

    def _run(self, cell_indexes: List[int], notebook_id: str,
             client_id: Optional[str] = None
             ) -> ExecuteNotebookCellsResult:
      try:
        if not client_id or client_id == "":
          kernel_result, _ = BootJupyterKernelTool().invoke({})
          client_id = json.loads(kernel_result)["client_id"]

        if notebook_id == client_id:
          raise ToolException(
              "Идентификаторы notebook_id и client_id должны отличаться.")

        uuid_version = 4
        uuid_obj = uuid.UUID(client_id, version=uuid_version)

        completion_message, result_json = (
            ExecuteNotebookCellsTool().invoke(
                {
                  "cell_indexes": cell_indexes,
                  "notebook_id": notebook_id,
                  "client_id": client_id,
                }
            )
        )

        logs_str = json.loads(result_json)["logs"]
        logs: str = f"""
        {completion_message}\n
        Идентификатор клиента Jupyter, client_id: {client_id}.
        Пожалуйста запомните его,
        так как он понадобится при последующих запусках ячеек,
        либо при остановке ядра!\n
        {logs_str}
        """
        return ExecuteNotebookCellsResult(logs="logs").model_dump_json()

      except ValueError as val_err:
        raise ToolException(f"""Проблема client_id.
        Для решения проблемы либо укажите верный и валидный идентификатор,
        либо не передавайте client_id вовсе.""")
      except Exception as exc:
        raise ToolException(f"Smart execution error: {str(exc)}")

In [27]:
# SmartExecuteCellsTool().invoke({
#     'cell_indexes': [0],
#     'notebook_id': '512deaf2-b476-4644-8386-7167cb4e15b5',
#     'client_id': '616d62bd-401d-491c-8a1c-3310a858e09d'})

## **Утилита изменения содержимого ячейки блокнота**

In [28]:
class UpdateCellInput(BaseModel):
  cells: list[dict[str, str]] = Field(description=(
      "Список ячеек, которые нужно изменить.\n"
      "Каждая ячейка - это словарь со следующими ключами:"
      "- index: целые неотрицательное число "
      "в виде строки (позиция вставки ячейки)"
      "- type: тип ячейки (строка) — либо 'code', либо 'markdown'.\n"
      "- source: содержимое ячейки (строка).\n\n"
      "Пример (для изменения ячейки с индексом 2):\n"
      "[{\"index\": '2', \"type\": \"code\", "
      "\"source\": \"import numpy as np\"}]"))

  notebook_id: str = Field(description=(
      "Строка формата UUID. Уникальный идентификатор Jupyter Notebook."))


class UpdateNotebookCellsTool(BaseTool):
  name: str = "update_cells_in_jupyter_notebook"
  description: str = (
      "Изменяет содержимое ячеек в Jupyter Notebook по индексам.\n\n"
      "Аргументы:\n"
      "- `cells`: список ячеек для обновления. "
      "Каждая ячейка - это словарь со следующими ключами: "
      "index - индекс ячейки, type - тип ячейки и "
      "source - содержимое ячейки.\n"
      "- `notebook_id`: строка формата UUID. "
      "Уникальный идентификатор Jupyter Notebook, "
      "который должен быть получен из инструмента LoadJupyterNotebook "
      "до выполнения действий с ячейками.\n\n"
      "После замены содержимого ячеек, "
      "изменения сохраняются в объекте Jupyter Notebook.\n"
      "Инструмент не выполняет ячейки — только обновляет содержимое."
  )
  args_schema: Type[BaseModel] = UpdateCellInput
  return_direct: bool = False
  # response_format: str = "content_and_artifact"

  def _run(self, cells: list[dict[str, str]], notebook_id: str) -> str:
    try:

      notebook: NotebookNode = artifact_store.get_notebook(notebook_id)

      for cell in cells:
        idx: int = int(cell.get("index"))
        if not( 0 <= idx < len(notebook.cells) ):
          raise ToolException(
              "Индекс ячейки вне допустимого диапазона."
              "Уточните количество ячеек, чтобы указать верные индексы."
          )

        type_new_cell: str = cell.get("type", "markdown")
        new_content: str = cell.get("source", "")

        if notebook.cells[idx].cell_type != type_new_cell:
          if type_new_cell == "code":
            updated_cell: NotebookNode = new_code_cell(
                source=new_content
            )
          else:
            updated_cell: NotebookNode = new_markdown_cell(
                source=new_content
            )
            notebook.cells[idx] = updated_cell
        else:
          notebook.cells[idx]['source'] = new_content

      return "Ячейки успешно изменены."

    except Exception as exc:
      raise ToolException(
          f"Error: Failed to update cell contents.\n{str(exc)}"
      )

In [29]:
# change_cell = UpdateNotebookCellsTool()

# print(change_cell.name)
# print(change_cell.description)
# print(change_cell.args)
# print(change_cell.return_direct)

# change_cell.invoke({
#     "cells": {2: {"type": "code", "source": "print(np.dot(x, y))"} },
#     "notebook_id": "1fe30d37-9a3d-4df3-bd5b-c9ba03840908",
#     })

## **Утилита добавления ячейки в блокнот**

In [30]:
class AddCellInput(BaseModel):
  cells: list[dict[str, str]] = Field(description=
   ("Список ячеек List, которые нужно добавить.\n"
   "Каждая ячейка - это словарь со следующими ключами:"
   "- index: целые неотрицательное число "
   "в виде строки (позиция вставки ячейки)"
   "- type: тип ячейки (строка) — либо 'code', либо 'markdown'.\n"
   "- source: содержимое ячейки (строка)."))

  notebook_id: str = Field(description=(
      "Строка формата UUID. Идентификатор Jupyter Notebook.\n"
      "Пример: '811ce62f-b2db-4169-867c-c682521007ef'"
      ))


class AddCellNotebookTool(BaseTool):
  name: str = "adding_cells_to_jupyter_notebook"
  description: str = (
      "Добавляет ячейки в Jupyter Notebook по заданным индексам.\n"
      "Параметры:\n"
      "- `cells`: список ячеек (List) для добавления. "
      "Каждая ячейка - это словарь со следующими ключами: "
      "index - индекс вставки, type - тип ячейки и "
      "source - содержимое ячейки.\n"
      "- `notebook_id`: строка формата UUID. "
      "Идентификатор Jupyter Notebook, "
      "который должен быть получен из инструмента LoadJupyterNotebook "
      "до выполнения действий с ячейками.\n\n"
      "Добавленные ячейки сохраняются в объекте Jupyter Notebook, "
      "но не выполняются."
  )
  args_schema: Type[BaseModel] = AddCellInput
  return_direct: bool = False

  def _run(self, cells: list[dict[str, str]], notebook_id: str) -> str:
    try:
      notebook: NotebookNode = artifact_store.get_notebook(notebook_id)

      for cell in cells:
        if not( 0 <= int(cell.get("index", -1)) <= len(notebook.cells) ):
          raise ToolException(
              "Индекс ячейки вне допустимого диапазона."
              "Уточните количество ячеек, чтобы указать верные индексы."
          )
        if cell.get("type") == "code":
          new_cell: NotebookNode = new_code_cell(
              source=cell.get("source", "")
          )
        else:
          new_cell: NotebookNode = new_markdown_cell(
              source=cell.get("source", "")
          )
        notebook.cells.insert(int(cell.get("index")), new_cell)

      return (f"Ячейки {cells} успешно добавлены в Jupyter Notebook "
      f"с идентификатором {notebook_id}.")

    except KeyError as key_exc:
      raise ToolException(f"Не существует блокнота с таким notebook_id.")
    except Exception as exc:
      raise ToolException(f"Error: Couldn't add cells.\n{str(exc)}")

In [31]:
# add_cell = AddCellNotebookTool()

# print(add_cell.name)
# print(add_cell.description)
# print(add_cell.args)
# print(add_cell.return_direct)

# add_cell.invoke({
#     'cells': [
#         {
#             'index': '12',
#             'type': 'code',
#             'source': 'import numpy as np\nvector = np.array([1, 2, 3])\nscalar = 5\nresult = vector * scalar\nprint("Original vector:", vector)\nprint("Scalar:", scalar)\nprint("Result:", result)'
#         },
#         # {
#         #     'index': '1',
#         #     'type': 'markdown',
#         #     'source': '## Converting a NumPy array to a Pandas Series\n\nYou can turn a 1‑D NumPy arrayinto a pandas Series using the `pd.Series` constructor:\n\n```python\nimport pandas as pd\nimport numpy as np\n\narr = np.array([10, 20, 30, 40])\nseries = pd.Series(arr)\n\nprint(series)\n```\n\nIf the NumPy array is multi‑dimensional, you can flatten it first or convert each sub‑array to a Series:\n\n```python\narr2d = np.array([[1,2], [3,4]])\nseries_of_arrays = pd.Series(arr2d)          # holds each row as an array\n```\n\nFor a 1‑D array, `pd.Series(arr)` is usually enough. Pandas will automatically assign integer indices if none are provided.'
#         # }
#     ],
#     'notebook_id': 'b769c30a-dcac-480d-be9d-dedf75f5c717'
# })

## **Утилита уточнения количества ячеек в блокноте**

In [32]:
class CountNotebookCellsInput(BaseModel):
  notebook_id: str = Field(description=(
      "Строка формата UUID. "
      "Уникальный идентификатор Jupyter Notebook."))


class CountNotebookCellsResult(BaseModel):
  count_cells: int = Field(description=(
      "Количество ячеек в Jupyter Notebook."))


class CountNotebookCellsTool(BaseTool):
  name: str = "get_the_number_of_cells_in_jupyter_notebook"
  description: str = (
    "Возвращает количество ячеек (целое число) "
    "в Jupyter Notebook по его идентификатору.\n\n"
    "Параметры:\n"
    "- `notebook_id`: строка формата UUID. "
    "Идентификатор Jupyter Notebook, который должен быть "
    "получен из инструмента LoadJupyterNotebook до "
    "выполнения действий с ячейками."
  )
  args_schema: Type[BaseModel] = CountNotebookCellsInput
  return_direct: bool = False
  response_format: str = "content_and_artifact"

  def _run(self, notebook_id: str) -> CountNotebookCellsResult:
    try:
      notebook: NotebookNode = artifact_store.get_notebook(notebook_id)
      content: str = f"В блокноте {len(notebook.cells)} ячеек."
      artifact = CountNotebookCellsResult(
          count_cells=len(notebook.cells)).model_dump_json()
      return content, artifact

    except Exception as exc:
      raise ToolException((
          "Error: Couldn't find out the number of cells "
          f"in the notebook.\n{str(exc)}"
      ))

In [33]:
# count_cell = CountNotebookCellsTool()

# print(count_cell.name)
# print(count_cell.description)
# print(count_cell.args)
# print(count_cell.return_direct)

# count_cell.invoke({ "notebook_id": "9ba651a0-cd80-4687-9a53-ac6480efc911" })

## **Утилита удаления ячейки блокнота**

In [34]:
class DeleteCellInput(BaseModel):
  cell_indexes: List[int] = Field(description=(
    "Список индексов ячеек (целые неотрицательные числа), "
    "которые нужно удалить. Индексы соответствуют текущему порядку "
    "ячеек в Jupyter Notebook."))
  notebook_id: str = Field(description=(
      "Строка формата UUID. Уникальный идентификатор Jupyter Notebook."))


class DeleteCellNotebookTool(BaseTool):
  name: str = "deleting_cells_in_jupyter_notebook"
  description: str = (
      "Удаляет ячейки по их индексам из Jupyter Notebook.\n\n"
      "Параметры:\n"
      "- `cell_indexes`: список целых чисел — "
      "индексы ячеек для удаления.\n"
      "- `notebook_id`: строка формата UUID. "
      "Идентификатор Jupyter Notebook, "
      "который должен быть получен из инструмента LoadJupyterNotebook "
      "до выполнения действий с ячейками.\n\n"
      "Удаление происходит по текущему состоянию Jupyter Notebook. "
      "Ячейки удаляются без выполнения. "
      "Изменения сохраняются в объекте Jupyter Notebook."
  )
  args_schema: Type[BaseModel] = DeleteCellInput
  return_direct: bool = False
  # response_format: str = "content_and_artifact"

  def _run(self, cell_indexes: List[int], notebook_id: str) -> str:
    try:
      notebook: NotebookNode = artifact_store.get_notebook(notebook_id)

      for idx in sorted(cell_indexes, reverse=True):
        if not( 0 <= idx < len(notebook.cells) ):
          raise ToolException(
              "Индекс ячейки вне допустимого диапазона."
              "Уточните количество ячеек, чтобы указать верные индексы."
          )
        notebook.cells.pop(idx)

      return "Ячейки успешно удалены."

    except Exception as exc:
      raise ToolException(f"Error: Couldn't delete cells.\n{str(exc)}")

In [35]:
# delete_cell = DeleteCellNotebookTool()

# print(delete_cell.name)
# print(delete_cell.description)
# print(delete_cell.args)
# print(delete_cell.return_direct)

# delete_cell.invoke({
#     "cell_indexes": [4],
#     "notebook_id": "1fe30d37-9a3d-4df3-bd5b-c9ba03840908",
#     })

## **Утилита сохранения изменений блокнота**

In [36]:
class SaveNotebookInput(BaseModel):
  notebook_id: str = Field(description=(
      "Строка формата UUID. Уникальный идентификатор Jupyter Notebook."))


class SaveNotebookTool(BaseTool):
  name: str = "save_jupyter_notebook_to_disk"
  description: str = (
      "Сохраняет изменения Jupyter Notebook в исходный .ipynb файл.\n\n"
      "Параметры:\n"
      "- `notebook_id`: строка формата UUID. "
      "Идентификатор Jupyter Notebook.\n\n"
      "Возвращает текстовую строку - успешность сохранения."
  )
  args_schema: Type[BaseModel] = SaveNotebookInput
  return_direct: bool = False

  def _run(self, notebook_id: str) -> str:
    try:
      path: str = artifact_store.get_notebook_path_file(
          notebook_id=notebook_id)
      notebook: NotebookNode = artifact_store.get_notebook(
          notebook_id=notebook_id)
      with open(path, "w", encoding="utf-8") as f:
        nbformat.write(notebook, f)
      return "Файл успешно сохранен."
    except Exception as exc:
      raise ToolException(f"Error: Couldn't save the file.\n{str(exc)}")

In [37]:
# save_notebook = SaveNotebookTool()

# print(save_notebook.name)
# print(save_notebook.description)
# print(save_notebook.args)
# print(save_notebook.return_direct)

# save_notebook.invoke({
#     "notebook_id": "1fe30d37-9a3d-4df3-bd5b-c9ba03840908",
#     })

## **Утилита просмотра содержимого ячеек**

In [38]:
class ViewContentNotebookCellsInput(BaseModel):
  cell_indexes: List[int] = Field(description=(
      "Список номеров ячеек из Jupyter Notebook, "
      "содержимое которых необходимо вывести в качестве результата.")
  )
  notebook_id: str = Field(description=(
      "Строка формата UUID. Уникальный идентификатор Jupyter Notebook.")
  )


class ViewContentNotebookCellsResult(BaseModel):
  content: str = Field(description=(
      "Строка с содержимым указанных ячеек, "
      "включая комментарии с номерами ячеек.")
  )


class ViewContentNotebookCellsTool(BaseTool):
  name: str = "display_contents_jupyter_notebook_cells"
  description: str = (
      "Отображает содержимое указанных ячеек "
      "Jupyter Notebook по их индексам.\n\n"
      "Параметры:\n"
      "- `cell_indexes`: список индексов ячеек.\n"
      "- `notebook_id`: строка формата UUID. "
      "Идентификатор Jupyter Notebook, "
      "который должен быть получен из инструмента LoadJupyterNotebook "
      "до выполнения действий с ячейками.\n\n"
      "Возвращает JSON-строку с многострочным текстом, "
      "содержащим исходное содержимое указанных ячеек."
  )
  args_schema: Type[BaseModel] = ViewContentNotebookCellsInput
  return_direct: bool = False

  def _run(self, cell_indexes: List[int], notebook_id: str
           ) -> ViewContentNotebookCellsResult:
    try:
      lines = []
      notebook: NotebookNode = artifact_store.get_notebook(notebook_id)

      for i in cell_indexes:
        if len(notebook.cells) == 0:
          raise ToolException(
              "В блокноте нет ячеек. "
              "Добавьте ячейки и повторите попытку снова."
          )
        if not( 0 <= i < len(notebook.cells) ):
          raise ToolException(
              "Индекс ячейки вне допустимого диапазона."
              "Уточните количество ячеек, чтобы указать верные индексы."
          )
        cell = notebook.cells[i]
        lines.append(f"# Содержимое ячейки №{i}\n{cell.source}")

      str_lines: str = "\n\n".join(lines)
      artifact = ViewContentNotebookCellsResult(
          content=str_lines).model_dump_json()
      return artifact

    except Exception as exc:
      raise ToolException(
          f"Error: Couldn't read the contents of the cells.\n{str(exc)}")

In [39]:
# view_cell = ViewContentNotebookCellsTool()

# print(view_cell.name)
# print(view_cell.description)
# print(view_cell.args)
# print(view_cell.return_direct)

# view_cell.invoke({
#     "cell_indexes": list(range(1)),
#     "notebook_id": "76501fee-5830-42e7-9ee4-cc6c693561fc",
#     })

## **Утилита завершения работы Jupyter Kernel**

In [40]:
class StopJupyterKernelInput(BaseModel):
  client_id: str = Field(description=(
      "Строка формата UUID. "
      "Идентификатор клиента, подключённого к ядру Jupyter.")
  )
  kernel_id: str = Field(description=(
      "Строка формата UUID. "
      "Идентификатор менеджера ядра Jupyter (KernelManager).")
  )


class StopJupyterKernelTool(BaseTool):
  name: str = "stopping_jupiter_core"
  description: str = (
    "Полностью завершает работу ядра Jupyter, "
    "остановив все вычисления и разорвав соединение с клиентом.\n\n"
    "Параметры:\n"
    "- `client_id`: строка формата UUID. "
    "Идентификатор клиента (BlockingKernelClient), "
    "подключённого к ядру.\n"
    "- `kernel_id`: строка формата UUID. "
    "Идентификатор менеджера ядра (KernelManager).\n\n"
    "Инструмент закрывает каналы связи с ядром и завершает его работу. "
    "Следует использовать только "
    "при необходимости полной остановки вычислений."
  )
  args_schema: Type[BaseModel] = StopJupyterKernelInput
  return_direct: bool = False

  def _run(self, client_id: str, kernel_id: str) -> str:
    try:
      kc: BlockingKernelClient = artifact_store.get_client(
          client_id=client_id)
      km: KernelManager = artifact_store.get_kernel_manager(
          kernel_id=kernel_id)
      kc.stop_channels()
      km.shutdown_kernel()
      return "Ядро Jupyter успешно остановлено. Все вычисления завершены."
    except Exception as exc:
      raise ToolException(f"""Error:
      An error occurred when stopping jupiter core.\n{str(exc)}""")

In [41]:
# stopping_jupiter = StopJupyterKernelTool()

# print(stopping_jupiter.name)
# print(stopping_jupiter.description)
# print(stopping_jupiter.args)
# print(stopping_jupiter.return_direct)

# stopping_jupiter.invoke({
#     "kernel_id":"9452e0cb-192a-4ae5-b9a1-d81088fad3af",
#     "client_id":"84387604-8599-45b8-8da6-4673e68da691"
#     })

## **Утилита веб-поиска**

In [42]:
class WebSearchInput(BaseModel):
  query: str = Field(description=(
      "Чёткий поисковый запрос или вопрос, например: "
      "'Как загрузить csv файл, используя pandas?'"))


class WebSearchResult(BaseModel):
  search_results: list[str] = Field(description="Список URL-адресов.")


class WebSearchTool(BaseTool):
  name: str = "web_search"
  description: str = (
      "Инструмент для поиска информации в интернете. "
      "Используется, когда нужно найти свежие или общедоступные данные "
      "по заданному текстовому запросу. "
      "Параметры:\n"
      "- `query`: Текст запроса.\n\n"
      "Инструмент возвращает список URL-адресов с "
      "релевантными результатами."
  )
  args_schema: Type[BaseModel] = WebSearchInput
  return_direct: bool = False

  def _run(self, query: str) -> WebSearchResult:
    try:
      max_links: int = 2
      url_list: list[str] = list()

      selection: list[dict[str, str]] = DDGS().text(query, max_results=5)

      for web_element in selection:
        url_list.append( web_element.get("href", "") )

      artifact = WebSearchResult(search_results=url_list
                                 ).model_dump_json()
      return artifact
    except Exception as exc:
      raise ToolException(
          f"Error: Couldn't make a web request.\n{str(exc)}")

In [43]:
# web_search = WebSearchTool()

# print(web_search.name)
# print(web_search.description)
# print(web_search.args)
# print(web_search.return_direct)

# web_search.invoke({'query': 'Как преобразовать массив numpy в pandas Series'})

## **Утилита извлечения текста из URL-ссылок**

### **Про фильтрацию контента с помощью LLM**

Можно использовать фильтрацию на основе LLM - [LLMContentFilter](https://docs.crawl4ai.com/core/markdown-generation/#53-llmcontentfilter)

Но нужно сверять [поддерживаемые](https://docs.litellm.ai/docs/providers) LiteLLM модели
с действующими моделями провайдера!

Также [поддерживаемые](https://docs.crawl4ai.com/api/parameters/) модели Crawl4AI

Вот пример:
```
from crawl4ai.content_filter_strategy import LLMContentFilter
from crawl4ai import DefaultMarkdownGenerator
browser_config = BrowserConfig(
    enable_stealth=True,  # Включение скрытного режима
    text_mode=True,       # Отключает загрузку изображений
)
filter = LLMContentFilter(
llm_config = LLMConfig(
    provider="groq/llama-3.3-70b-versatile",
    api_token="env: GROQ_API_KEY"),
instruction=(
    "## Ограничение\n"
    "1) Вам запрещается выдумывать какую либо информацию.\n"
    "2) Вы должны следовать согласно указаниям ниже:\n\n"
    "## Задача"
    "Удали информацию не релевантную  запросу из текста.\n\n"
    "## Запрос\n"
    "Как преобразовать массив numpy в pandas.Series?\n\n"
    "## Формат ответа\n"
    "В качестве ответа следует вернуть строку, "
    "представляющую собой отредактированную версию текста.\n\n"
    "## Обработка исключений\n"
    "Если текст полностью не соответствует запросу или "
    "содержит информацию о проверке что Вы не робот, "
    "то верните пустую строку - ''."
),
chunk_token_threshold=4096,
verbose=True
)
md_generator = DefaultMarkdownGenerator(
content_filter=filter,
options={"ignore_links": True}
)
exclusion_tags: list[str] = [
'form', 'head', 'header', 'footer', 'nav', 'title', 'script',
'a', 'img', 'ul', 'ol', 'button', 'hr', 'label', 'input',
'aside', 'fieldset', 'form', 'video', 'source'
]
run_config = CrawlerRunConfig(
markdown_generator=md_generator,
excluded_tags=exclusion_tags,   # Удалить целые блоки
remove_forms=True,              # Удаляет веб-формы
exclude_external_links=True,    # Удалить внешние ссылки
exclude_social_media_links=True,# Удалить ссылки на соц. сети
remove_overlay_elements=True,   # Remove popups/modals
)
```



---



### **Код извлечения контента**

In [44]:
PROMPT_FOR_EXTRACTION = PromptTemplate.from_template(
    "## Ограничение\n"
    "1) Вам запрещается выдумывать какую либо информацию.\n"
    "2) Вы должны следовать согласно указаниям ниже:\n\n"
    "## Задача"
    "Удали информацию не релевантную  запросу.\n\n"
    "## Запрос\n"
    "{query}\n\n"
    "## Информация\n"
    "{content}\n\n"
    "## Формат ответа\n"
    "В качестве ответа следует вернуть строку, "
    "представляющую собой отредактированную версию текста "
    "или пустую строку в виде исключительной ситуации.\n\n"
    "## Обработка исключений\n"
    "Если текст полностью не соответствует запросу или "
    "содержит информацию о проверке что Вы не робот, "
    "то верните пустую строку - ''."
)

class ExtractingDataFromLinkUrlInput(BaseModel):
  url_list: list[str] = Field(description=(
      "Список URL-адресов веб-страниц, "
      "с которых нужно извлечь текстовую информацию."))
  query: str  = Field(description=(
      "Вопрос, в виде строки, на который нужно найти ответ в сети."))


class ExtractingDataFromLinkUrlTool(BaseTool):
  name: str = "extract_information_from_web_links"
  description: str = (
      "Извлекает текст из веб-страниц по переданным URL и "
      "сохраняет его в векторное хранилище "
      "для последующего поиска и анализа.\n\n"
      "Параметры:\n"
      "- `url_list`: список URL-адресов для обработки.\n\n"
      "- `query`: вопрос для поиска информации в сети.\n\n"
      "Возвращает строку с результатом извлечения."
  )
  args_schema: Type[BaseModel] = ExtractingDataFromLinkUrlInput
  return_direct: bool = False

  async def parsing_task_generator(self, urls: list[str]):
    tasks = [asyncio.create_task(self.parser(url)) for url in urls]
    for task in asyncio.as_completed(tasks):
      url, suc, cont = await task
      yield url, suc, cont

  def _run(self, url_list: list[str], query: str) -> str:
    asyncio.run(self._arun(url_list=url_list, query=query))

  async def _arun(self, url_list: list[str], query: str) -> str:
    # store = get_store()
    # user_id = get_config().get("configurable").get("user_id")
    # namespace = ("memories", user_id)

    processed = 0
    skipped = []
    async for url, success, content in self.parsing_task_generator(urls=url_list):
      try:
        if not success:
          skipped.append((url, "Empty content"))
          continue

        cleaned_up_content = llm.invoke(
            PROMPT_FOR_EXTRACTION.format(query=query, content=content)
        )

        if cleaned_up_content.content:
          self._save_to_vectorstore(
              # store=store,
              # namespace=namespace,
              text=cleaned_up_content.content,
              metadata={"source": url},
          )
        processed += 1

      except Exception as exc:
          raise ToolException((
              "Error: Couldn't extract information from websites.\n"
              f"{str(exc)}"))

    result = f"Успешно обработано: {processed} ссылок."
    if skipped:
      result += f" Пропущено: {len(skipped)} ссылок. Подробнее: {skipped}"
    return result

  async def parser(self, url: str):
    browser_config = BrowserConfig(
        enable_stealth=True,  # Включение скрытного режима
        text_mode=True,       # Отключает загрузку изображений
    )
    exclusion_tags: list[str] = [
        'form', 'head', 'header', 'footer', 'nav', 'title', 'script',
        'a', 'img', 'ul', 'ol', 'button', 'hr', 'label', 'input',
        'aside', 'fieldset', 'form', 'video', 'source'
    ]
    run_config = CrawlerRunConfig(
        excluded_tags=exclusion_tags,   # Удалить целые блоки
        remove_forms=True,              # Удаляет веб-формы
        exclude_external_links=True,    # Удалить внешние ссылки
        exclude_social_media_links=True,# Удалить ссылки на соц. сети
        remove_overlay_elements=True,   # Remove popups/modals
    )
    async with AsyncWebCrawler(config=browser_config) as crawler:
      result = await crawler.arun(
          url=url,
          config=run_config)
      return url, result.success, result.markdown


  # LlamaIndex подход:
  # https://developers.llamaindex.ai/python/examples/vector_stores/chromaindexdemo
  def _save_to_vectorstore(self, text: str, metadata: dict) -> None:
    parser = MarkdownNodeParser()
    documents = [
        Document(
            text=text,
            metadata=metadata
        )
    ]
    nodes = parser.get_nodes_from_documents(documents)
    index = VectorStoreIndex(
        nodes=nodes,
        storage_context=storage_context,
    )


  # LangChain подход
  # def _save_to_vectorstore(
  #     self,
  #     store: BaseStore,
  #     namespace: tuple[str, str],
  #     text: str,
  #     metadata: dict,
  # ) -> None:
  #   document = Document(
  #       page_content=text,
  #       metadata=metadata,
  #       id=str(uuid.uuid4())
  #   )

  #   doc_splits = text_splitter.split_documents([document])

  #   for doc in doc_splits:
  #     key = str(uuid.uuid4())
  #     value = {
  #         "document": doc.page_content,
  #         "metadata": doc.metadata,
  #     }
  #     store.put(namespace, key, value)

In [45]:
# extracte_data = ExtractingDataFromLinkUrlTool()

# print(extracte_data.name)
# print(extracte_data.description)
# print(extracte_data.args)
# print(extracte_data.return_direct)
# extracte_data.tool_call_schema.model_json_schema()

# extracte_data.invoke(
#     {'url_list': [
#         "https://progler.ru/blog/preobrazovanie-massiva-numpy-v-seriyu-pandas",
#         # "https://statorials.org/ru/серия-pandas-таблица-numpy/",
#         # "https://blog.finxter.com/5-best-ways-to-convert-a-numpy-array-to-a-pandas-series/",
#     ],
#     'query': 'Как преобразовать массив numpy в pandas.Series?',
# }
# )

## **Утилита для поиска данных в векторном хранилище**

In [46]:
class SearchInMemoryToolInput(BaseModel):
  query: str = Field(description=(
      "Запрос, по которому нужно найти информацию "
      "в векторном хранилище."))


class SearchInMemoryToolResult(BaseModel):
  search_results: list[dict] = Field(description=(
      "Список словарей с ключами:\n"
      "key - идентификатор элемента в хранилище;\n"
      "text - содержимое элемента;\n"
      "metadata - дополнительная информация об элементе.\n"
      "Данный список возвращает элементы из хранилища, "
      "соответствующие запросу."))

class SearchInMemoryTool(BaseTool):
  name: str = "search_in_memory"
  description: str = (
      "Осуществляет поиск релевантной информации в локальной памяти "
      "(векторном хранилище), используя "
      "семантическое соответствие запросу."
      "\n\nПараметры:\n"
      "- `query`: Запрос, на который нужно найти "
      "информацию в хранилище.\n\n"
      "Возвращает список документов, которые соответствуют запросу.")
  args_schema: Type[BaseModel] = SearchInMemoryToolInput
  return_direct: bool = False

  def _run(self, query: str) -> SearchInMemoryToolResult:
    try:
      # Поиск в памяти
      store = get_store()
      user_id = get_config().get("configurable").get("user_id")
      namespace = ("memories", user_id)
      number_sources: int = 5
      results: list[dict] = (
          store.search(
              namespace=namespace,
              query=query,
              limit=number_sources,
          )
      )

      # Поиск в RAG
      index: VectorStoreIndex = VectorStoreIndex.from_vector_store(
          vector_store,
      )
      query_engine: BaseQueryEngine = index.as_query_engine()
      response = query_engine.query(query)
      for node in response.source_nodes:
        results.append({
            "text": response.response,
            "metadata": node.node.metadata
        })

      # Формирование результирующего ответа
      artifact = SearchInMemoryToolResult(
          search_results=results).model_dump_json()
      return artifact
    except Exception as exc:
        raise ToolException(
            "Error: Couldn't extract information from vector storage.\n"
            f"{str(exc)}")

In [47]:
# search_mem_tool = SearchInMemoryTool()
# search_mem_tool.invoke({
#     "query": "Как преобразовать numpy массив в pandas.Series?"
# })

## **Инструмент комплексного поиска информации в различных источниках**

In [48]:
# TODO: Нужно переписать на рабочий процесс LlamaIndex, по примеру отсюда:
# https://developers.llamaindex.ai/python/examples/workflow/corrective_rag_pack/

class SearchWebAndMemoryTool(BaseTool):
    name: str = "search_web_and_memory"
    description: str = (
        "Поиск по интернету с автоматической загрузкой данных в память и "
        "последующим семантическим поиском по векторному хранилищу."
        "Использует инструменты web_search, "
        "extract_information_from_web_links и search_in_memory."
        "Параметры:\n"
        "- `query`: Текст запроса.\n\n"
    )
    args_schema: Type[BaseModel] = WebSearchInput
    return_direct: bool = False

    def _run(self, query: str) -> SearchInMemoryToolResult:
      try:
        search_json = WebSearchTool().invoke({"query": query})
        url_list = json.loads(search_json)["search_results"]

        ExtractingDataFromLinkUrlTool().invoke({
            "url_list":url_list,
            "query":query
        })

        search_results = SearchInMemoryTool().invoke({"query": query})
        search_results_dict = json.loads(search_results)["search_results"]
        return SearchInMemoryToolResult(search_results=search_results_dict
                                        ).model_dump_json()
      except Exception as exc:
          raise ToolException(f"Search error: {str(exc)}")

In [49]:
# search_web_and_mem = SearchWebAndMemoryTool()
# search_web_and_mem.invoke(
#     {'query': 'Как преобразовать массив numpy в pandas.Series?',}
# )

# **Оформление вывода ответа агента**

In [50]:
separator = 30*"="
console = Console(width=64)
def print_readable_chunk(chunk):
  if not chunk:
    return
  for msg in chunk.get("messages", []):
    if isinstance(msg, HumanMessage):
        console.print((
            "\n🧑 Пользователь:\n"
            f"{msg.content}"
        ))
    elif isinstance(msg, AIMessage):
      console.print((
          "\n🤖 Ассистент:\n"
          f"{msg.content}\n"
      ))
      if msg.tool_calls:
        for tool_call in msg.tool_calls:
          console.print((
              "🔧 Вызов инструмента:\n"
              f"{tool_call['name']} с аргументами\n"
              f"{tool_call['args']}\n"
          ))
    elif isinstance(msg, ToolMessage):
      syntax = msg.content
      # syntax = Syntax(
      #     msg.content,
      #     "python",
      #     theme="monokai",
      #     line_numbers=True,
      #     code_width=64,
      # )
      console.print(f"📦 Результат инструмента ({msg.name}):\n")
      console.print(syntax)
    else:
        console.print((
            "❓ Неизвестный тип сообщения:\n"
            f"{msg}\n"
        ))
    console.print(separator)

# **Инициализация агента**

## **Задание инструментов**

In [51]:
tools: list = [
    # CreateNotebookTool(),
    # LoadJupyterNotebookTool(),
    OpenOrCreateNotebookTool(),

    BootJupyterKernelTool(),
    ExecuteNotebookCellsTool(),
    # SmartExecuteCellsTool(),

    UpdateNotebookCellsTool(),
    AddCellNotebookTool(),
    DeleteCellNotebookTool(),
    CountNotebookCellsTool(),
    SaveNotebookTool(),
    ViewContentNotebookCellsTool(),
    StopJupyterKernelTool(),

    # WebSearchTool(),
    # ExtractingDataFromLinkUrlTool(),
    # SearchInMemoryTool(),
    SearchWebAndMemoryTool(),
]
tools_by_name = {tool.name: tool for tool in tools}

## **Задание промпта**

In [52]:
system_message_template = SystemMessagePromptTemplate.from_template("""
## Роль
Вы являетесь профессиональным программистом в сфере data science
и machine learning.

## Аудитория
С вами общаются программистом из тех же областей, но, возможно,
с меньшим опытом работы в этой области. Отвечайте им на русском языке.
Вот прошлые факты, которые могут оказаться полезными:
{long_term_context}

## Действия
Вы должны обрабатывать датасеты и обучать модели нейронных сетей
в Jupyter Notebook на языке Python 3 и
использовать соответствующие библиотеки для анализа данных в своей работе.
Ваши размышления должны быть тщательными, поэтому не будет ничего страшного,
если они будут очень длинными. Вы должны продумывать действия шаг за шагом
до и после каждого действия, которое вы решите предпринять.
Обращайтесь к пользователю только в критических ситуациях.
Вы ДОЛЖНЫ выполнять итерацию и продолжать, пока проблема не будет решена.

## Примерная цепочка работы с блокнотом
1) Загрить блокнот;
2) Работа с ячеками:
2.1) Выполнение, добавление, просмотр и удаление.
Перед выполнением каких либо действий с ячейками нужно
уточнить текущее количетсво ячеек через
get_the_number_of_cells_in_jupyter_notebook!
2.2) Выполнить необходимо действие.
3) Сохранить блокнот;
4) Остановить ядро Jupyter. Если произвелась остановка ядра, то при дальшейшем
выполнении ячеек (если этого требует пользователь)
нужно заново запустить ядро - инструмент booting_jupyter_kernel!

## Инструменты
Вам доступен ряд инструментов для выполнения вашей работы. Они включают
набор инструментов для работы с Jupyter Notebook и инструменты для поиска
информации в интернет сети или в векторном хранилище.
Вот список инструментов:
{tools_name}

### Важная информация об используемых параметрах:
notebook_id - это строковый идентификатор загруженного блокнота.
client_id - это строковый идентификатор клиента, который подключен к блокноту.
kernel_manager_id - это строковый идентификатор ядра блокнота,
который выполняет код в ячейках.
Все три идентификатора: notebook_id, client_id и kernel_manager_id
можно получить выполнив загрузку блокнота
с помощью инструмента open_or_create_notebook!

## Выходные данные
В результате вашей работы вы должны написать готовый и полностью рабочий
код для обработки датасета, или обучения нейросети, или полный цикл
пайплайн разработки нейромоделей. Весь код должен быть написан, выполнен
и сохранен в Jupyter Notebook.

## Настроение и стиль ответов
Ваши ответы должны быть понятны как начальному разработчику, так и с опытом.
Используй для ответов неформальный стиль общения, как в повседневной жизни.

## обработка исключений
Если ответ на вопрос пользователя не в указанном контексте, говори,
что ты можешь отвечать только на вопросы о программировании.
""")

# **Построение графа ии-агента**

Материалы по реализации памяти агента:
1) https://redis.io/blog/langgraph-redis-build-smarter-ai-agents-with-memory-persistence/
2) https://redis.io/blog/build-smarter-ai-agents-manage-short-term-and-long-term-memory-with-redis/
3) https://medium.com/@anil.jain.baba/long-term-agentic-memory-with-langgraph-824050b09852
4) https://saptak.in/writing/2025/03/23/mastering-long-term-agentic-memory-with-langgraph
5) https://blog.futuresmart.ai/how-to-build-langgraph-agent-with-long-term-memory
6) https://supermemory.ai/blog/3-ways-to-build-llms-with-long-term-memory/

### Инициализация класса-состояния графа

In [53]:
"""
Можно использовать в качестве состояния MessagesState, но TypedDict более гибок.
Подробнее про MessagesState:
https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate
"""
class State(TypedDict):
  """
  Класс определяющий хранение истории сообщений.

  возможно, intermediate_steps (если хочешь помнить все вызовы инструментов)
  необязательно: agent_scratchpad, tools_used, loop_count и пр.
  MessagesPlaceholder(variable_name="agent_scratchpad")
  """
  messages: Annotated[list[AnyMessage], add_messages]
  summary: str

### Создание узла выполнения инструментов

In [54]:
def tool_node(state: State) -> dict:
  result = []
  for tool_call in state["messages"][-1].tool_calls:
    tool_name = tool_call["name"]

    if tool_name not in tools_by_name:
      error_message = (
          f"Ошибка: инструмент '{tool_name}' не найден. "
          "Пожалуйста, введите правильное название инструмента"
      )
      result.append(
          ToolMessage(
              content=error_message,
              name=tool_name,
              tool_call_id=tool_call["id"]
          )
      )
      continue

    tool = tools_by_name[tool_name]
    observation = tool.invoke(tool_call["args"])
    result.append(
        ToolMessage(
            content=observation,
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        )
    )
  return {"messages": result}
# tool_node = ToolNode(tools)

### Функции условий

In [55]:
# https://docs.langchain.com/oss/python/langgraph/workflows-agents#routing
def llm_call_router(state: State):
  messages = state["messages"]
  last_message = messages[-1]
  if last_message.tool_calls:
    return "tools"
  return "save_memory"



---



### Узел получения информации из памяти

In [56]:
def retrieve_memories(
    state: State,
    config: RunnableConfig,
    store: BaseStore) -> dict:
  """
  Семантический поиск информации.
  """
  user_id = config["configurable"]["user_id"]
  namespace = ("memories", user_id)
  history = state["messages"]
  last_message = history[-1].content
  memories = store.search(namespace=namespace, query=last_message)
  info = "\n".join([d.get("text", "") for d in memories])

  # print(f"Результат поиска в хранилище:\n{info}")
  return {"summary": info}

### Узел сохранения информации в память

In [57]:
def update_memory(
    state: State,
    config: RunnableConfig,
    store: BaseStore) -> dict:
    user_id = config["configurable"]["user_id"]
    namespace = (user_id, "memories")
    last_messages = state["messages"]
    summary = state.get("summary", "")

    if summary:
      analyze_prompt = HumanMessage(
          content=(
              "Это краткое изложение беседы на данный момент: "
              "Дополните краткое изложение, "
              "приняв во внимание новые сообщения, приведенные выше:"
          )
      )
    else:
      analyze_prompt = HumanMessage(
          content=(
              "Составьте краткое изложение приведенного выше разговора:"
          )
      )
    analyze_msg = llm.invoke(last_messages + [analyze_prompt])

    memory_content = analyze_msg.content.strip()
    memory_id = str(uuid.uuid4())
    store.put(
        namespace=namespace,
        key=memory_id,
        value={
            "document": memory_content,
            "metadata": {"source": "Фрагмент из памяти"}
        },
    )

    return {"summary": analyze_msg.content, "messages": [analyze_msg]}

### Узел вызова LLM

In [58]:
def call_model(state: State, config: RunnableConfig) -> dict:
  history = state["messages"]
  recovered_memories = state.get("summary", "")

  system_message = system_message_template.format(
      long_term_context=recovered_memories,
      tools_name=tools_by_name,
  )
  template = ChatPromptTemplate.from_messages([
      system_message,
      MessagesPlaceholder(variable_name="messages"),
  ])

  prompt = template.invoke({"messages": history})
  llm_with_tools = llm.bind_tools(tools)
  response = llm_with_tools.invoke(prompt.to_messages())
  return {"messages": [response]}

### Конструирование графа

In [59]:
checkpointer = InMemorySaver()
# Семантический поиск
# store = InMemoryStore(
#     index={
#         "dims": 1536,
#         "embed": embeddings,
#         "fields": ["facts"],
#     }
# )

builder = StateGraph(State)

# Регистрируем узлы
builder.add_node("fetch_memory", retrieve_memories)
builder.add_node("save_memory", update_memory)

builder.add_node("chat", call_model)
builder.add_node("tools", tool_node)

# Определяем условия переходов между узлами
builder.add_conditional_edges(
    "chat",
    llm_call_router,
    {
        "tools": "tools",
        "save_memory": "save_memory"
    },
)
builder.add_edge(START, "fetch_memory")
builder.add_edge("fetch_memory", "chat")
builder.add_edge("tools", "chat")
builder.add_edge("save_memory", END)

# Строим граф
graph = builder.compile(
    checkpointer=checkpointer,
    store=store,
)

### Визуализация графа

In [60]:
# https://mermaid.live/edit
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
        __start__([<p>__start__</p>]):::first
        fetch_memory(fetch_memory)
        save_memory(save_memory)
        chat(chat)
        tools(tools)
        __end__([<p>__end__</p>]):::last
        __start__ --> fetch_memory;
        chat -.-> save_memory;
        chat -.-> tools;
        fetch_memory --> chat;
        tools --> chat;
        save_memory --> __end__;
        classDef default fill:#f2f0ff,line-height:1.2
        classDef first fill-opacity:0
        classDef last fill:#bfb6fc

### Вызов агента

In [61]:
"""
Задачи:
1) Создай новый блокнот (директория создания - '/content') и ячейку, в которой будет происходить умножение вектора (из любых 3 целых чисел) на число;
2) Найти информацию как преобразовать массив numpy в pandas Series;
3) На основании полученной информации добавь ячейку в блокнот;
4) Выполни ячейки и сохрани изменения.
==============================================
2) Найти информацию как преобразовать массив numpy в pandas Series;
3) На основании полученной информации добавь ячейку в блокнот;
==============================================
Добавь в существующий блокнот код для создания столбцатой диаграммы.
Данные для диаграммы - это массив numpy, который уже имеется в ячейке.
Подписи к записям пусть будут в виде: "Элемент №порядковый_номер".
Не забудь сохранить изменения!
"""

# Необходимо для реализации Persistence
config = {"configurable": {"thread_id": "main", "user_id": "user-1"}}

while True:
  user_input: str = input("Ваше сообщение:\n")

  stop_words = ("выход", "конец", "exit", "close")
  stop_signal = False
  for word in stop_words:
    if word in user_input.lower():
      stop_signal = True

  if stop_signal:
    break

  human_message = {
    "messages": [
        HumanMessage(
            content=(user_input)
        )
    ]
  }

  for chunk in graph.stream(human_message, config=config):
    for role, content in chunk.items():
      print_readable_chunk(content)

Ваше сообщение:
Задачи: 1) Создай новый блокнот (директория создания - '/content') и ячейку, в которой будет происходить умножение вектора (из любых 3 целых чисел) на число; 2) Найти информацию как преобразовать массив numpy в pandas Series; 3) На основании полученной информации добавь ячейку в блокнот; 4) Выполни ячейки и сохрани изменения.


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 39.0MiB/s]
2025/11/05 17:59:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'ToolV2' object has no attribute 'get'


🤖 Ассистент:
Я создам новый блокнот в директории '/content' и добавлю ячейку 
с кодом для умножения вектора на число. Затем я найду информацию
о том, как преобразовать массив numpy в pandas Series. На основе
этой информации я добавлю новую ячейку в блокнот. После этого я 
выполню обе ячейки и сохраню изменения в блокноте.

🔧 Вызов инструмента:
open_or_create_notebook с аргументами
{'file_name': 'vector_multiplication.ipynb', 'path': '/content'}

==============================

...

==============================

2025/11/05 18:00:53 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'ToolV2' object has no attribute 'get'


🤖 Ассистент:
Я выполнил все задачи:

1. Создал новый блокнот с именем 'vector_multiplication.ipynb' в
директории '/content' и добавил ячейку с кодом для умножения 
вектора на число.
2. Найдено информацию о том, как преобразовать массив numpy в 
pandas Series.
3. Добавил новую ячейку в блокнот с кодом для преобразования 
массива numpy в pandas Series.
4. Выполнил обе ячейки и сохранил изменения в блокноте.

Блокнот успешно сохранен с именем 'vector_multiplication.ipynb' 
в директории '/content'.

==============================

TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '8ca2d34ff5e3120325267f7be88e6e24', 'date': 'Wed, 05 Nov 2025 18:00:55 GMT', 'x-envoy-upstream-service-time': '7', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '4472d2f1-fbcf-4b06-be0d-94b39e080ebf', 'message': "You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@